In [44]:
import pandas as pd
import re

def cleaning(s):
  s = str(s)
  s = re.sub(', ', ',', s)
  s = re.sub(',', ' ', s)
  return s

df = pd.read_csv("./datasets/final.csv", encoding="utf-8")
df = df.dropna()
text_data = open('./datasets/slogans.txt', 'w', encoding="utf-8")
for idx, item in df.iterrows():
  slogans = cleaning(item['company']) + ', ' + item['slogan']+ '\n'
  text_data.write(slogans)
text_data.close()

In [45]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast


def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
                bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                pad_token='<pad>', mask_token='<mask>')
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir, legacy_format=False)
   
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

train_file_path = './datasets/slogans.txt'
model_name = 'skt/kogpt2-base-v2'
output_dir = './models2'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

C:\Users\tyler\anaconda3\envs\AI_exam\lib\site-packages\transformers\data\datasets\language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
 49%|████▉     | 500/1020 [01:43<01:47,  4.83it/s]

{'loss': 2.4771, 'learning_rate': 2.5490196078431373e-05, 'epoch': 2.45}


 98%|█████████▊| 1000/1020 [03:27<00:04,  4.88it/s]

{'loss': 1.6098, 'learning_rate': 9.80392156862745e-07, 'epoch': 4.9}


100%|██████████| 1020/1020 [03:33<00:00,  4.77it/s]


{'train_runtime': 213.6555, 'train_samples_per_second': 4.774, 'epoch': 5.0}


In [46]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "./models2"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence},', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.pad_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

# sequence = input()
# max_len = int(input())

input = '오픈월드 게임'

sequence = input
max_len = 50

print('input :' + sequence)

for i in range(10):
    print(generate_text(sequence, max_len))

input :오픈월드 게임
오픈월드 게임, 내가 하고 싶은 이야기를 찾아<eos>테일즈런너, 모바일 RPG 게임내 손에서 시작하는 새로운 MMORPG<eos>테라M, 모바일 MMORPG 게임M
None
오픈월드 게임, 한국 이용자를 위한 최고의 선택<eos>리니지2, 리니지2 레볼루션, 리니지2의 탄생<eos>리니지2, 리니지2레볼루션, 리
None
오픈월드 게임, 한국을 넘어 세계인의 게임 축제<eos>2016서울모터쇼2016, 자동차 문화축제하늘 아래 어디든 자유롭게<eos>2018서울모터쇼2018, 자동차 문화축제꿈이
None
오픈월드 게임, 대한민국 유저의 게임상대가 되다<eos>넷마블 모바일 전략 RPG세상이 부른 영웅들<eos>넷마블 MMORPG대한민국의 영웅들의 모바일 전쟁<
None
오픈월드 게임, 한국 게임당신의 모든 판타지 모험<eos>라그나로크 오리진, 모바일 MMORPG 게임나만의 곳으로 떠나<eos>라그나로크 오리진, 모바일 MMORP
None
오픈월드 게임, 그리고 퍼블리셔가 되다<eos>스마일게이트 메가박스, 프리미엄 콘텐츠 제공업겨울은 로맨틱하게<eos>스마일게이트 탭소닉, 프리미엄 콘텐츠
None
오픈월드 게임, 한국의 모든 유저는 WITH MOVE<eos>KT기가 IOT, 인공지능 홈 IoT 서비스아이들이 IOT 하니까 기가 IOT<eos>
None
오픈월드 게임, 대한민국을 하나로 이어가다<eos>일곱 개의대죄, 영웅들이여.여러분이 원하던 세상<eos>어드벤처, SF 모바일 RPG 게임영웅들이여, 이
None
오픈월드 게임, 엔터테인먼트 게임새롭게 만나는 새로운 세계<eos>아발론 모바일 레이드 업데이트, 모바일 게임즐거움을 위한 모험<eos>아발론 모바일 레이드 업데이트,
None
오픈월드 게임, 대한민국에 없던 세계<eos>삼국블레이드, 모바일 MMORPG 게임한 번은 삼국지다, 한번은 삼국지다<eos>삼국블레이드2, 모바일 MMORPG 게임
None


In [8]:
import requests
import json

company = "어쌔씬크리드 발할라" #(최대 10글자)
context = "오픈월드 게임" #(최대 15글자)

input = company + ',' + context + ','

r = requests.post(
    'https://train-nqhy6iii8xyt5739th8f-gpt2-train-teachable-ainize.endpoint.ainize.ai/predictions/gpt-2-ko-small-finetune', #기업설명, 슬로건
    headers = {'Content-Type' : 'application/json'
               },
    data=json.dumps({
  "text": input,
  "num_samples": 10,
  "length": 20
    }))

#print(r.json())
print('input :' + input)
print('='*50)
for slogan in r.json():
    slogan = slogan.split('\n')[0]
    slogan = slogan.split(',')[2:]
    slogan = ', '.join(slogan)
    if slogan :
        print(slogan)
        print('='*50)

input :어쌔씬크리드 발할라,오픈월드 게임,
새로운 시작이 시작된다 
모든 게 신기루처럼 
다함께 
월드에 없던 새로운 즐거움이 생겼다 
MMORPG 게임 
새로운 세계 
이토록 긴박한 모험은 한번도 경험해보지 못한 
모바일 대전 게임에서 가장 큰 즐거움이 있다 
새로운 모험의 시작 
당신만 아는 녀석 
